In [1]:
# pip install --upgrade torch

In [1]:
## Libraries
import pandas as pd
import numpy as np
import pickle
import warnings
import copy
import random
import matplotlib.pyplot as plt
from collections import Counter
import joblib

In [2]:
## Classifiers considered in the Ensemble model (Surrogate Model)
# Trained Decision Tree Classifier
DT = joblib.load("portscan_DT.joblib")
# Trained MLP Classifier
MLP = joblib.load("portscan_MLP.joblib")
# Trained Logistic Regression Classifier
Logit = joblib.load("portscan_logit.joblib")

In [3]:
## Attack data to be uploaded
df_all = pd.read_csv("portscan_train.csv")

In [34]:
## Specify the attack type
attack_type = "Port Scan"

In [35]:
df = df_all[df_all['Label'] == attack_type]

In [5]:
df

,eth_header,ip_header,tcp_header,tcp_options,tcp_segment_data,Label
0,0019b90a69f100c1b114eb310800,4500003c82a740003e064329ac100001c0a80a32,932c1f4031e2c9bb00000000a0027210631b0000,020405b40402080a02c24b1c0000000001030307,00,Port Scan
2,0019b90a69f100c1b114eb310800,4500002cdb4f00002e063a91ac100001c0a80a32,f3b902d210349552000000006002040081280000,020405b4,0000,Port Scan
4,0019b90a69f100c1b114eb310800,4500003c447940003e068157ac100001c0a80a32,9bc2048264180cf400000000a002721002640000,020405b40402080a02c2488d0000000001030307,00,Port Scan
9,0019b90a69f100c1b114eb310800,4500003cb84340003e060d8dac100001c0a80a32,c4600e997effe60d00000000a0027210c5ec0000,020405b40402080a02c25e4e0000000001030307,00,Port Scan
10,0019b90a69f100c1b114eb310800,4500002cc46100002706587fac100001c0a80a32,a2b507b4b342614d00000000600204005e410000,020405b4,0000,Port Scan
...,...,...,...,...,...,...
227671,0019b90a69f100c1b114eb310800,4500003c8d2b40003e0638a5ac100001c0a80a32,b73e034ba1d4c3d900000000a0027210ed170000,020405b40402080a02c24ef20000000001030307,00,Port Scan
227672,0019b90a69f100c1b114eb310800,4500002c8a9f000039068041ac100001c0a80a32,ac010386051f35870000000060020400330d0000,020405b4,0000,Port Scan
227677,0019b90a69f100c1b114eb310800,4500002cbfb300002306612dac100001c0a80a32,e7c0c0f83062f79600000000600204004c880000,020405b4,0000,Port Scan
227679,0019b90a69f100c1b114eb310800,4500002c7b0800002806a0d8ac100001c0a80a32,8e6007c05af0c3f8000000006002040068310000,020405b4,0000,Port Scan


In [6]:
df.drop(columns = ["Label"], axis=1, inplace=True)

<ipython-input-6-5468986e0044>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ["Label"], axis=1, inplace=True)


In [7]:
df = df[[ 'eth_header', 'ip_header', 'tcp_header','tcp_options', 'tcp_segment_data']]

In [8]:
columns = [ 'eth_header', 'ip_header', 'tcp_header','tcp_options', 'tcp_segment_data']

In [9]:
df['all_payload'] = df['eth_header'] + df['ip_header'] + df['tcp_header'] + df['tcp_options'] + df['tcp_segment_data']

In [10]:
df

,eth_header,ip_header,tcp_header,tcp_options,tcp_segment_data,all_payload
0,0019b90a69f100c1b114eb310800,4500003c82a740003e064329ac100001c0a80a32,932c1f4031e2c9bb00000000a0027210631b0000,020405b40402080a02c24b1c0000000001030307,00,0019b90a69f100c1b114eb3108004500003c82a740003e...
2,0019b90a69f100c1b114eb310800,4500002cdb4f00002e063a91ac100001c0a80a32,f3b902d210349552000000006002040081280000,020405b4,0000,0019b90a69f100c1b114eb3108004500002cdb4f00002e...
4,0019b90a69f100c1b114eb310800,4500003c447940003e068157ac100001c0a80a32,9bc2048264180cf400000000a002721002640000,020405b40402080a02c2488d0000000001030307,00,0019b90a69f100c1b114eb3108004500003c447940003e...
9,0019b90a69f100c1b114eb310800,4500003cb84340003e060d8dac100001c0a80a32,c4600e997effe60d00000000a0027210c5ec0000,020405b40402080a02c25e4e0000000001030307,00,0019b90a69f100c1b114eb3108004500003cb84340003e...
10,0019b90a69f100c1b114eb310800,4500002cc46100002706587fac100001c0a80a32,a2b507b4b342614d00000000600204005e410000,020405b4,0000,0019b90a69f100c1b114eb3108004500002cc461000027...
...,...,...,...,...,...,...
227671,0019b90a69f100c1b114eb310800,4500003c8d2b40003e0638a5ac100001c0a80a32,b73e034ba1d4c3d900000000a0027210ed170000,020405b40402080a02c24ef20000000001030307,00,0019b90a69f100c1b114eb3108004500003c8d2b40003e...
227672,0019b90a69f100c1b114eb310800,4500002c8a9f000039068041ac100001c0a80a32,ac010386051f35870000000060020400330d0000,020405b4,0000,0019b90a69f100c1b114eb3108004500002c8a9f000039...
227677,0019b90a69f100c1b114eb310800,4500002cbfb300002306612dac100001c0a80a32,e7c0c0f83062f79600000000600204004c880000,020405b4,0000,0019b90a69f100c1b114eb3108004500002cbfb3000023...
227679,0019b90a69f100c1b114eb310800,4500002c7b0800002806a0d8ac100001c0a80a32,8e6007c05af0c3f8000000006002040068310000,020405b4,0000,0019b90a69f100c1b114eb3108004500002c7b08000028...


In [11]:
payload_1 = '474554202f3f785857333d6b646872646552622668575479446c445054723d436b596f374b266565506879713d4354677171533356367750574d6247266e506b4a633d446c78413772676c734362426720485454502f312e310d0a4163636570742d456e636f64696e673a202a2c20677a69700d0a486f73743a203230352e3137342e3136352e36380d0a4b6565702d416c6976653a203536340d0a4163636570742d436861727365743a207574662d382c49534f2d383835392d313b713d302e302c2a3b713d302e390d0a436f6e6e656374696f6e3a206b6565702d616c6976650d0a43616368652d436f6e74726f6c3a206d61782d6167653d300d0a557365722d4167656e743a204d6f7a696c6c612f352e30202857696e646f7773204e5420362e333b20574f57363429204170706c655765624b69742f3533352e333620284b48544d4c2c206c696b65204765636b6f292056657273696f6e2f352e302e34205361666172692f3533372e31300d0a436f6e74656e742d547970653a206170706c69636174696f6e2f782d75726c2d656e636f6465640d0a0d0a'
payload_2 = '474554202f3f33616965376267383d4d305232414626746837414e423d714f516631717664746120485454502f312e310d0a486f73743a203230352e3137342e3136352e36380d0a4163636570742d456e636f64696e673a20677a69700d0a4b6565702d416c6976653a203638310d0a557365722d4167656e743a204d6f7a696c6c612f352e30202857696e646f7773204e5420362e313b20574f57363429204765636b6f2f32303035323531312046697265666f782f31332e300d0a4163636570742d436861727365743a2049534f2d383835392d322c7574662d383b713d302e312c2a3b713d302e330d0a436f6e6e656374696f6e3a206b6565702d616c6976650d0a526566657265723a20687474703a2f2f7777772e79616e6465782e636f6d2f774a46776b733f676f52423d4b4a320d0a43616368652d436f6e74726f6c3a206e6f2d63616368650d0a436f6e74656e742d547970653a206170706c69636174696f6e2f782d75726c2d656e636f6465640d0a0d0a'
payload_3 = '474554202f3f646d62326f67627935503d4831336d3550684a33697563565851266947466734333d4d5331655837694746756e504555624a556820485454502f312e310d0a4163636570742d456e636f64696e673a206964656e746974792c20677a69700d0a436f6e6e656374696f6e3a206b6565702d616c6976650d0a4b6565702d416c6976653a203435350d0a557365722d4167656e743a204d6f7a696c6c612f352e30202857696e646f77733b20553b204d53494520362e313b2057696e646f7773204e5420362e313b202e4e455420434c5220322e352e393137393b2057696e36343b20783634290d0a4163636570742d436861727365743a207574662d382c49534f2d383835392d313b713d302e332c2a3b713d302e370d0a486f73743a203230352e3137342e3136352e36380d0a526566657265723a20687474703a2f2f7777772e62696e672e636f6d2f4456656e460d0a43616368652d436f6e74726f6c3a206e6f2d63616368650d0a436f6e74656e742d547970653a206170706c69636174696f6e2f782d75726c2d656e636f6465640d0a0d0a'

In [12]:
## Transforms decimal to hexadecimal
def transformed_hex_preprocess(decimal):
    
    hexa = hex(decimal).lstrip("0x")
    if (len(hexa) % 2) != 0:
        hexa = '0' + hexa
    return hexa

In [13]:
def hex_norm(payload, col):
    dict_hex_cols = {'ip_header':20 , 'tcp_header': 20 , 'tcp_options': 40, 'tcp_segment_data': 1460 }
    byte_array = bytes.fromhex(payload)
    byte_lst = list(byte_array)
    if (len(byte_lst) < dict_hex_cols[col]):
            output = np.pad(byte_lst, (0, dict_hex_cols[col]-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:dict_hex_cols[col]].copy())
            
    #normalize       
    output = np.abs(output.astype(float)) / 255
    return output

In [14]:
def concat_dfs(df1, df2, df3, df4):
    packet_p = pd.DataFrame([df1]).add_prefix('ip_header_byte_')
    packet_t = pd.DataFrame([df2]).add_prefix('tcp_header_byte_')
    packet_n = pd.DataFrame([df3]).add_prefix('tcp_options_byte_')
    packet_x = pd.DataFrame([df4]).add_prefix('tcp_segment_data_byte_')
    
    packet_purturbed = pd.concat([packet_p , packet_t , packet_n, packet_x ], axis =1)
    
    return packet_purturbed

In [15]:
def df_preprocess(df):
    
    df.drop(columns=['ip_header_byte_9','ip_header_byte_0', 'ip_header_byte_1'],inplace=True) #remove ipv4 byte, differentiated service filed, and protocol bytes
    packet_purturbed_final = df.drop(df.loc[:, 'ip_header_byte_12':'tcp_header_byte_3'].columns, axis=1) #remove src and dest ips and ports bytes
    return packet_purturbed_final

In [16]:
## Calculates the ip checksum
def calc_checksum(packet):
    pkt = copy.deepcopy(packet)
    checksum_header = pkt['ip_header'][:20] + pkt['ip_header'][24:]
    sum_ = 0
    for i in range(0,len(checksum_header),4):
        h = checksum_header[i] + checksum_header[i+1] + checksum_header[i+2] + checksum_header[i+3]
        sum_ += int(h,16)
    
    ch = hex(sum_).lstrip("0x")

    if len(ch) % 4 != 0:
        ch_ = '000'+ ch
    
    check_ = int(ch_[:4],16) + int(ch_[4:],16)
    check_hex = hex(check_).lstrip("0x")

    if len(check_hex) > 4:
        for i in range(8-len(check_hex)):
            check_hex = '0' + check_hex
        check_hex = transformed_hex_preprocess(int(check_hex[:4],16) + int(check_hex[4:],16))

    check_sum_dec = int('ffff',16) - int(check_hex,16)

    check_sum = hex(check_sum_dec).lstrip("0x")
    
    if check_sum_dec != 0:
        if len(check_sum) % 4 != 0:
            for d in range(4-len(check_sum)%4):
                check_sum = '0' + check_sum
    else:
        check_sum = '0000'

    return check_sum

In [17]:
# https://www.securitynik.com/2015/08/calculating-udp-checksum-with-taste-of_3.html

In [18]:
## Calculates TCP checksum
def calc_tcp_checksum(packet):
    pkt = copy.deepcopy(packet)
    tcp_checksum_header = pkt['tcp_header'][0:32] + pkt['tcp_header'][36:]
    sum_header = 0
    for i in range(0,len(tcp_checksum_header),4):
        h1 = tcp_checksum_header[i] + tcp_checksum_header[i+1] + tcp_checksum_header[i+2] + tcp_checksum_header[i+3]
        sum_header += int(h1,16)
    tcp_options = pkt['tcp_options']
    if len(tcp_options) % 4 != 0:
        for i in range(4-len(tcp_options) % 4):
            tcp_options = '0' + tcp_options
    sum_options = 0
    for i in range(0,len(tcp_options),4):
        h2 = tcp_options[i] + tcp_options[i+1] + tcp_options[i+2] + tcp_options[i+3]
        sum_options += int(h2,16)
    
    tcp_payload = pkt['tcp_segment_data']
    if len(tcp_payload) % 4 != 0:
        int_fg = len(tcp_payload)// 4
        pad = ''
        for a in range(4 - len(tcp_payload) % 4):
            pad += '0'
        tcp_payload = tcp_payload[: int_fg*4] + pad + tcp_payload[int_fg*4:]

    
    sum_payload = 0
    for i in range(0,len(tcp_payload),4):
        h3 = tcp_payload[i] + tcp_payload[i+1] + tcp_payload[i+2] + tcp_payload[i+3]
        sum_payload += int(h3,16)
        
    if len(pkt['tcp_segment_data']) <= 2:
        tcp_segment_len = 0
    elif len(pkt['tcp_segment_data']) >= 2920:
        tcp_segment_len = 2920 // 2
    else:
        tcp_segment_len = len(pkt['tcp_segment_data'])// 2

    if len(pkt['tcp_options']) <= 2:
        tcp_options_len = 0
    else:
        tcp_options_len = len(pkt['tcp_options'])// 2
    
    
    
    total_length_ = int(pkt['ip_header'][4:8],16) - 20 
    
    
    tcp_check_dec = int(pkt['ip_header'][24:28],16) + int(pkt['ip_header'][28:32],16) + int(pkt['ip_header'][32:36],16)\
            + int(pkt['ip_header'][36:],16)  + total_length_ + sum_header + sum_options+ sum_payload + 6 
    
    
    ch = hex(tcp_check_dec).lstrip("0x")
    
    if len(ch) % 4 != 0:
        for j in range(4-len(ch)%4):
            ch = '0'+ ch
    
        
    check_ =  int(ch[4:],16) + int(ch[:4],16)
    check_hex = hex(check_).lstrip("0x")
    if len(check_hex) > 4:
        for i in range(8-len(check_hex)):
            check_hex = '0' + check_hex
        check_hex = transformed_hex_preprocess(int(check_hex[:4],16) + int(check_hex[4:],16))

    check_sum_dec = int('ffff',16) - int(check_hex,16)
    
    check_sum = hex(check_sum_dec).lstrip("0x")
    
    if check_sum_dec > 0:
        if len(check_sum) % 4 != 0:
            for d in range(4-len(check_sum)%4):
                check_sum = '0' + check_sum
    else:
        check_sum = '0000'
    return check_sum

In [19]:
## Implements the action fragmentation given a payload
## Action 1
def action_frag(pkt, payload):
    packet = copy.deepcopy(pkt)
    #frag is allowed ()
    packet['ip_header'] = packet['ip_header'][:12] + "00" + packet['ip_header'][14:]
    
    #change in length
    added_length = len(payload)//2 
    old_length_dec = int(packet['ip_header'][4:8], 16)
    new_length_dec = added_length + old_length_dec
    new_length_hex = transformed_hex_preprocess(new_length_dec)
    if len(new_length_hex) < 4 :
        new_length_hex  = '00'+ new_length_hex
    
    packet['ip_header']=packet['ip_header'][:4] + new_length_hex + packet['ip_header'][8:]
    
    #reduce ttl by 1
    old_ttl_dec = int(packet['ip_header'][16:18], 16)
    new_ttl_dec = old_ttl_dec - 1
    if new_ttl_dec <= 1:
        new_ttl_dec = 1
    new_ttl_hex = transformed_hex_preprocess(new_ttl_dec)
    packet['ip_header']=packet['ip_header'][:16] + new_ttl_hex + packet['ip_header'][18:]
    
    #IP checksum update
    new_checksum_hex = calc_checksum(packet)
    packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]

    #segment data update
    if packet['tcp_segment_data'] == '00':
         packet['tcp_segment_data']= payload
            
    else:
         packet['tcp_segment_data']=packet['tcp_segment_data'] + payload
            
    ## TCP Checksum update
    new_tcp_checksum_hex = calc_tcp_checksum(packet)
    packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]

    return packet

In [20]:
## Implements the action more fragmentation 
## Action 2
def action_more_frag(pkt, payload):
    packet = copy.deepcopy(pkt)
    #frag is allowed ()
    packet['ip_header']= packet['ip_header'][:12] + "20" + packet['ip_header'][14:]
    
    #change in length
    added_length = len(payload)//2 
    old_length_dec = int(packet['ip_header'][4:8], 16)
    new_length_dec = added_length + old_length_dec
    new_length_hex = transformed_hex_preprocess(new_length_dec)
    if len(new_length_hex) < 4 :
        new_length_hex  = '00'+ new_length_hex
    packet['ip_header']=packet['ip_header'][:4] + new_length_hex + packet['ip_header'][8:]
    
    #reduce ttl by 1
    old_ttl_dec = int(packet['ip_header'][16:18], 16)
#     print(old_ttl_dec)

    new_ttl_dec = old_ttl_dec - 1
    if new_ttl_dec <= 1:
        new_ttl_dec = 1
        
#     print(new_ttl_dec)
    new_ttl_hex = transformed_hex_preprocess(new_ttl_dec)
#     print('new_ttl_hex',new_ttl_hex)
    packet['ip_header']=packet['ip_header'][:16] + new_ttl_hex + packet['ip_header'][18:]
    
#     print(len(packet['ip_header']))
    #IP checksum update
    new_checksum_hex = calc_checksum(packet)
#     print('new_checksum_hex',new_checksum_hex)
    packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]

    #segment data update
    if packet['tcp_segment_data'] == '00':
         packet['tcp_segment_data']= payload
            
    else:
         packet['tcp_segment_data']=packet['tcp_segment_data'] + payload
            
    ## TCP Checksum update
    new_tcp_checksum_hex = calc_tcp_checksum(packet)
    packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]

    return packet

In [21]:
## Implements the action increasing or decreasing TTL values
## Action 3
def action_ttl(pkt,change_amount):
    packet = copy.deepcopy(pkt)
    
    #change ttl by change amount
    old_ttl_dec = int(packet['ip_header'][16:18], 16)
    new_ttl_dec = old_ttl_dec + change_amount
    
    if new_ttl_dec > 255:
        new_ttl_dec = 255
    elif new_ttl_dec <= 0:
        new_ttl_dec = 1
    new_ttl_hex = transformed_hex_preprocess(new_ttl_dec)
    packet['ip_header']=packet['ip_header'][:16] + new_ttl_hex + packet['ip_header'][18:]
    
    #IP checksum update
    new_checksum_hex = calc_checksum(packet)
    packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]
    
    return packet

In [22]:
## Implements the action increasing or decreasing window size
## Action 4
def action_window_size(pkt,change_amount):
    packet = copy.deepcopy(pkt)
    
    # Change window size by change amount
    old_window_dec = int(packet['tcp_header'][28:32],16)
    new_window_dec = old_window_dec + change_amount
    
    if new_window_dec > 65535:
        new_window_dec = 65535
    elif new_window_dec <= 0:
        new_window_dec = 1
        
    new_window_hex  = transformed_hex_preprocess(new_window_dec)
    
    
    if len(new_window_hex) < 4:
        for i in range(4 - len(new_window_hex)):
            new_window_hex  = '0' + new_window_hex

    packet['tcp_header'] = packet['tcp_header'][:28] + new_window_hex + packet['tcp_header'][32:]
    
    # TCP Checksum update
    new_tcp_checksum_hex = calc_tcp_checksum(packet)
    packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]
    
    return packet

In [23]:
## Implements the action addition of payload to the TCP segment 
## Action 5
def action_add_payload(pkt,payload):
    packet = copy.deepcopy(pkt)

    #segment data update
    if packet['tcp_segment_data'] == '00':
         packet['tcp_segment_data']= payload
            
    else:
         packet['tcp_segment_data']=packet['tcp_segment_data'] + payload
            
    # IP header total length change 
    added_length = len(payload)//2 
    old_length_dec = int(packet['ip_header'][4:8], 16)
    new_length_dec = added_length + old_length_dec
    new_length_hex = transformed_hex_preprocess(new_length_dec)
    if len(new_length_hex) < 4 :
        new_length_hex  = '00'+ new_length_hex
    packet['ip_header']=packet['ip_header'][:4] + new_length_hex + packet['ip_header'][8:]
   
    # IP Checksum update
    new_checksum_hex = calc_checksum(packet)

    packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]

    
    ## TCP Checksum update
    new_tcp_checksum_hex = calc_tcp_checksum(packet)
    
    packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]

    return packet

In [24]:
## Adds / changes Maximum segment size in TCP options
## Action 6
def action_MSS(pkt,change):
    packet = copy.deepcopy(pkt)
    
    str_ = packet['tcp_options']
    
    
    mss_start_index = str_.find('0204')
    
    
    if mss_start_index != -1:
        og_mss_hex = str_[mss_start_index+4 : mss_start_index+8 ]
    else:
        og_mss_hex = '0000'
    
    
    og_mss_dec = int(og_mss_hex,16)
    new_mss_dec = og_mss_dec + change
    if new_mss_dec < 0:
        new_mss_dec = 5
    
    new_mss_hex = transformed_hex_preprocess(new_mss_dec)
    
    
    if len(new_mss_hex) < 4:
        new_mss_hex = '00' + new_mss_hex
    
        
    if mss_start_index != -1:
        
        str_ = str_[:mss_start_index+4] + new_mss_hex + str_[mss_start_index+8:]
        
        packet['tcp_options'] = str_
        
        ## Change in checksum
        new_tcp_checksum_hex = calc_tcp_checksum(packet)
        packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]
        
    else:
        str_ = str_ + '01' + '01' + '02' + '04' + new_mss_hex + '01' + '01' 
        
        packet['tcp_options'] = str_
        
        ## Chnage in IP header total length
        added_length = 8
        old_length_dec = int(packet['ip_header'][4:8], 16)
        new_length_dec = added_length + old_length_dec
        new_length_hex = transformed_hex_preprocess(new_length_dec)
        if len(new_length_hex) < 4 :
            new_length_hex  = '00'+ new_length_hex
        packet['ip_header']=packet['ip_header'][:4] + new_length_hex + packet['ip_header'][8:]
        
        ## Change in IP header checksum
        new_checksum_hex = calc_checksum(packet)
        packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]
        
        ## Change in TCP header length
        old_len_dec = int(packet['tcp_header'][24:26],16)
        new_len_dec = old_len_dec + added_length
        new_len_hex = transformed_hex_preprocess(new_len_dec)
        packet['tcp_header'] = packet['tcp_header'][:24] + new_len_hex + packet['tcp_header'][26:]
        
        ## Change in TCP Checksum
        new_tcp_checksum_hex = calc_tcp_checksum(packet)
        packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]
    
    return packet

In [25]:
## Adds/changes window scale in TCP options
## Action 7
def action_windowscale(pkt,change):
    packet = copy.deepcopy(pkt)
    
    str_ = packet['tcp_options']
    
    
    scale_start_index = str_.find('0303')
    
    
    if scale_start_index != -1:
        og_scale_hex = str_[scale_start_index+4 : scale_start_index+6 ]

    else:
        og_scale_hex = '00'
    
    
    og_scale_dec = int(og_scale_hex,16)
    new_scale_dec = og_scale_dec + change
    if new_scale_dec <= 0:
        new_scale_dec = 1
    
    new_scale_hex = transformed_hex_preprocess(new_scale_dec)


    if scale_start_index != -1:
        
        str_ = str_[:scale_start_index+4] + new_scale_hex + str_[scale_start_index+6:]
        
        packet['tcp_options'] = str_
        
        ## Change in checksum
        new_tcp_checksum_hex = calc_tcp_checksum(packet)
        packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]
        
    else:
        str_ = str_ + '01' + '01' + '03' + '03' + new_scale_hex + '01' + '01' 
        
        packet['tcp_options'] = str_
        
        ## Chnage in IP header total length
        added_length = 7
        old_length_dec = int(packet['ip_header'][4:8], 16)
        new_length_dec = added_length + old_length_dec
        new_length_hex = transformed_hex_preprocess(new_length_dec)
        if len(new_length_hex) < 4 :
            new_length_hex  = '00'+ new_length_hex
        packet['ip_header']=packet['ip_header'][:4] + new_length_hex + packet['ip_header'][8:]
        
        ## Change in IP header checksum
        new_checksum_hex = calc_checksum(packet)
        packet['ip_header']=packet['ip_header'][:20] + new_checksum_hex + packet['ip_header'][24:]
        
        ## Change in TCP header length
        old_len_dec = int(packet['tcp_header'][24:26],16)
        new_len_dec = old_len_dec + added_length
        new_len_hex = transformed_hex_preprocess(new_len_dec)
        packet['tcp_header'] = packet['tcp_header'][:24] + new_len_hex + packet['tcp_header'][26:]
        
        ## Change in TCP Checksum
        new_tcp_checksum_hex = calc_tcp_checksum(packet)
        packet['tcp_header'] = packet['tcp_header'][0:32] + new_tcp_checksum_hex + packet['tcp_header'][36:]
    
    return packet

In [26]:
## Payload information chosen to be added while fragmenting
random.seed(1)
st_ind = random.randint(1,700)
payload1 = payload_1[st_ind : st_ind+20]

In [27]:
## Actions

# 1 -> Fragmentation and addition of payload 1
# 2 -> More fragmentation and addition of paylod 2

# 3-> Increase Time to live by 1
# 4 -> Decrease Time to live by 1

# 5 -> Increase window size by 10
# 6 -> Decrease window size by 10



# 7 -> Add Tcp options MSS (+10)
# 8 -> Add Tcp options MSS (-10)

# 9 -> Add Tcp options window Scale (+1)
# 10 -> Add Tcp options window (-1)

# 11 -> Addition of payload


In [28]:
## Given raw payload information this function outputs noramlized values and 1525 features
def state_extractor(packet):
    
    ip_header_dec_lst = hex_norm(packet['ip_header'], 'ip_header')
    tcp_header_dec_lst = hex_norm(packet['tcp_header'], 'tcp_header')
    tcp_option_dec_lst = hex_norm(packet['tcp_options'], 'tcp_options')
    tcp_segment_dec_lst = hex_norm(packet['tcp_segment_data'], 'tcp_segment_data')
    
    pkt_df = concat_dfs(ip_header_dec_lst, tcp_header_dec_lst, tcp_option_dec_lst, tcp_segment_dec_lst)
    
    final_pkt_df = df_preprocess(pkt_df)
    
    return final_pkt_df

In [29]:
## Checks if a packet is SYN/SYN-ACK
def syn_check(state):
    syn_ = False
    if state['tcp_header'][26:28] == '02' or state['tcp_header'][26:28] == '12':
        syn_ = True
    return syn_ 

In [30]:
## Random Action Function that makes an informed choice 
def random_action(state):
    if state['tcp_header'][26:28] == '02' or state['tcp_header'][26:28] == '12':
        action = random.randint(0,10)
    else:
        ac_li = [i for i in range(0,11) if i != 6 and i != 7 and i != 8 and i != 9]
        action = random.choice(ac_li)
    return action

In [31]:
## The ensemble mode which outputs the prediction based on unanimous voting and incremental reward
def ensemble_model(st):
    DT_predict = DT.predict(st)
    MLP_predict = MLP.predict(st)
    Logit_predict = Logit.predict(st)
    
    pred_list = [DT_predict[0], MLP_predict[0], Logit_predict[0]]
    
    if pred_list.count("Benign") == 1:
        reward = 50
        prediction = attack_type
    elif pred_list.count("Benign") == 2:
        reward = 300
        prediction = attack_type
    elif pred_list.count("Benign") == 3:
        reward = 600
        prediction = "Benign"
    else:
        reward = 0
        prediction = attack_type
        
    
    return prediction, reward

In [32]:
## This function takes the state and action as an input and outputs the next state and reward
def step(state,action,payload):
    done = False
    ## Fragmentation actions
    ## Frag
    if action == 0:
        next_state = action_frag(state,payload1)
        add_rew = -5
    ## More frag
    elif action == 1:
        next_state = action_more_frag(state,payload1)
        add_rew = -5
        
    ## TIme to live actions
    # Increase TTL
    elif action == 2:
        next_state = action_ttl(state,4)
        add_rew = -1
    # Decrease
    elif action == 3:
        next_state = action_ttl(state,-4)
        add_rew = -1
    
    ## Window Size actions
    # Increase
    elif action == 4:
        next_state = action_window_size(state,40)
        add_rew = -1
    # Decrease
    elif action == 5:
        next_state = action_window_size(state,-40)
        add_rew = -1
    
    ## Maximum segement size actions
    # Increase
    elif action == 6:
        if syn_check(state):
            next_state = action_MSS(state,40)
            add_rew = -5
        else:
            done = True
    # Decrease
    elif action == 7:
        if syn_check(state):
            next_state = action_MSS(state,-40)
            add_rew = -5
        else:
            done = True
    
    ## window scale
    # Increase
    elif action == 8:
        if syn_check(state):
            next_state = action_windowscale(state,4)
            add_rew = -5
        else:
            done = True
    # Decrease
    elif action == 9:
        if syn_check(state):
            next_state = action_windowscale(state,-4)
            add_rew = -5
        else:
            done = True
            
    ## Payload addition
    elif action == 10:
        next_state = action_add_payload(state,payload)
        add_rew = -1
    
    
    if done != True:
        next_state_df = state_extractor(next_state)

        class_pred,reward_ = ensemble_model(next_state_df)
        warnings.filterwarnings("ignore", category=UserWarning)

        if class_pred != attack_type:
            ## Reward for success
            reward = reward_ + add_rew
            done = True
            next_state = np.array([[1 for i in range(1525)]])

        else:
            ## Reward for failed attempt
            reward = add_rew + reward_
            done = False
            
    else:
        ## Infeasibility Reward
        reward = -1000
        next_state = np.array([[1 for i in range(1525)]])

    return next_state, reward, done

In [33]:
## Crafting the final state after done to be saved in the experience replay
example_ = df.iloc[9]
example_df = state_extractor(example_)
c_n = list(example_df.columns)
ar = np.full((1,1525),1)
final_state_df = pd.DataFrame(ar,columns = c_n)